In [1]:
from aimet_torch.cross_layer_equalization import equalize_model
from torchvision import models
import torch
import MobileNetV2Pretrained
import os
import sys

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)

from utils.input_getter import mug_image_getter

# Load a pretrained model

image_getter = mug_image_getter()
mug_image = image_getter.get_input_torch()
print(mug_image.shape)

modelOriginal   = MobileNetV2Pretrained.PreprocessedMobileNetV2(64, r"model/mobilenet_v2_coco.pth")
modelQuantized  = MobileNetV2Pretrained.PreprocessedMobileNetV2(64, r"model/mobilenet_v2_coco.pth")

modelOriginal.eval()
modelQuantized.eval()

# model = models.resnet18(pretrained=True)
# model = torch.nn.Sequential(
#     torch.nn.Flatten(),
#     torch.nn.Linear(3 * 224 * 224, 1024),
#     torch.nn.ReLU(),
#     torch.nn.Linear(1024, 512),
#     torch.nn.ReLU(),
#     torch.nn.Linear(512, 256),
#     torch.nn.ReLU()
# )


print(modelQuantized)
print(modelOriginal)

2025-02-26 14:52:00,024 - root - INFO - AIMET


/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/usr/lib/python3.10/abc.py:106: FutureWarning: `NLLLoss2d` has been deprecated. Please use `NLLLoss` instead as a drop-in replacement and see https://pytorch.org/docs/main/nn.html#torch.nn.NLLLoss for more details.
  cls = super().__new__(mcls, name, bases, namespace, **kwargs)


torch.Size([1, 3, 224, 224])
PreprocessedMobileNetV2(
  (mobilenet_v2): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(


In [2]:
from utils import helper
from dataset import utils as dsutils

helper.print_probablities_from_output(modelOriginal(mug_image), dsutils.GLOBAL_CLASSES)
helper.print_probablities_from_output(modelQuantized(mug_image), dsutils.GLOBAL_CLASSES)

# print(modelOriginal(mug_image))
# print(modelQuantized(mug_image))

Top-5 predictions for  on :
31 Cup                   78.5%
34 Bowl                   6.4%
37 Orange                 1.2%
33 Spoon                  1.2%
41 Donut                  0.9%
Top-5 predictions for  on :
31 Cup                   78.5%
34 Bowl                   6.4%
37 Orange                 1.2%
33 Spoon                  1.2%
41 Donut                  0.9%


In [4]:

param_num = 0

# print(list(model.parameters())[param_num])
# print(list(model.parameters())[param_num].shape)

# Apply Cross-Layer Equalization (CLE)
# This modifies the model in place

input_shape = (1, 3, 224, 224)
dummy_input = torch.randn(input_shape)

print("CLE Started")

equalize_model(modelQuantized, dummy_input = dummy_input)

print("Cross-Layer Equalization applied successfully!")

print(modelQuantized)

CLE Started
2025-02-26 14:52:14,899 - ConnectedGraph - WARNING - Unable to isolate model outputs.
Cross-Layer Equalization applied successfully!
PreprocessedMobileNetV2(
  (mobilenet_v2): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): Identity()
        (2): ReLU()
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
            (1): Identity()
            (2): ReLU()
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
          (2): Identity()
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1))
            (1): Identity()
            (2): ReLU()
          )
          (1): Conv2dN

In [5]:
helper.print_probablities_from_output(modelOriginal(mug_image), dsutils.GLOBAL_CLASSES)
helper.print_probablities_from_output(modelQuantized(mug_image), dsutils.GLOBAL_CLASSES)

Top-5 predictions for  on :
31 Cup                   78.5%
34 Bowl                   6.4%
37 Orange                 1.2%
33 Spoon                  1.2%
41 Donut                  0.9%
Top-5 predictions for  on :
31 Cup                   94.5%
34 Bowl                   1.8%
35 Banana                 0.4%
37 Orange                 0.3%
33 Spoon                  0.2%


In [ ]:
# Fix posle nemam pojma kako radi

# from aimet_torch.bias_correction import correct_bias
# import torch

# # Define your model (example: ResNet18)
# model = models.resnet18(pretrained=True)

# # Define a dummy dataset (batch size = 1, shape = [1, 3, 224, 224])
# dummy_input = torch.rand(1, 3, 224, 224)

# # Apply Bias Correction
# correct_bias(model)

# print("Bias Correction applied successfully!")

In [6]:
from aimet_torch.quantsim import QuantizationSimModel
from aimet_common.defs import QuantScheme
from torchvision import transforms
from torch.utils.data import DataLoader

# Dummy input to define the model input size
dummy_input = torch.rand(1, 3, 224, 224)

# Step 1: Create QuantizationSimModel
sim = QuantizationSimModel(modelQuantized, dummy_input=dummy_input,
                                     quant_scheme=QuantScheme.post_training_tf_enhanced,
                                     default_param_bw=8, default_output_bw=8)

from dataset import DatasetReader

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(ROOT_DIR)

datasetCOCO = DatasetReader.COCODataset(
    annotation_file = r"/home/centar15-desktop1/LPCV_2025_T1/datasets/coco/annotations/instances_val2017.json", 
    image_dir= r'/home/centar15-desktop1/LPCV_2025_T1/datasets/coco/val2017',
    target_classes=[s.lower() for s in dsutils.GLOBAL_CLASSES],
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]))

dataloader = DataLoader(datasetCOCO, batch_size=1, shuffle=True)

print(len(dataloader))



# Step 2: Compute Encodings (calibration)
def calibration_function(model, eval_iterations = 500, use_cuda = False):
    for i, data in enumerate(dataloader):
        image, _, _ = data
        model(image)
        if (i > eval_iterations):
            break
        if(i % 50 == 0):
            print(i)
    print(eval_iterations)
    print("cal")


2025-02-26 14:52:25,676 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-02-26 14:52:25,714 - Quant - INFO - No config file provided, defaulting to config file at /home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/aimet_common/quantsim_config/default_config.json
2025-02-26 14:52:25,726 - Quant - INFO - Unsupported op type Squeeze
2025-02-26 14:52:25,727 - Quant - INFO - Unsupported op type Mean
2025-02-26 14:52:25,733 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default
Total annotations before filtering: 36781
Total annotations after filtering: 13691
Total COCO annotations: 36781
Filtered annotations: 13691
13691


In [7]:
print(sim.model)

PreprocessedMobileNetV2(
  (mobilenet_v2): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): QuantizedConv2d(
          3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
          (param_quantizers): ModuleDict(
            (weight): QuantizeDequantize(shape=(), qmin=-128, qmax=127, symmetric=True)
            (bias): None
          )
          (input_quantizers): ModuleList(
            (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
          )
          (output_quantizers): ModuleList(
            (0): None
          )
        )
        (1): Identity()
        (2): QuantizedReLU(
          (param_quantizers): ModuleDict()
          (input_quantizers): ModuleList(
            (0): None
          )
          (output_quantizers): ModuleList(
            (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
          )
        )
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): 

In [8]:
sim.compute_encodings(calibration_function)

0
50
100
150
200
250
300
350
400
450
500
500
cal


In [9]:
print(sim.model)

PreprocessedMobileNetV2(
  (mobilenet_v2): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): QuantizedConv2d(
          3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
          (param_quantizers): ModuleDict(
            (weight): QuantizeDequantize(shape=(), qmin=-128, qmax=127, symmetric=True)
            (bias): None
          )
          (input_quantizers): ModuleList(
            (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
          )
          (output_quantizers): ModuleList(
            (0): None
          )
        )
        (1): Identity()
        (2): QuantizedReLU(
          (param_quantizers): ModuleDict()
          (input_quantizers): ModuleList(
            (0): None
          )
          (output_quantizers): ModuleList(
            (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
          )
        )
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): 

In [10]:
sim.model(mug_image)

DequantizedTensor([[-0.3103,  0.3103, -0.9828, -0.6724, -0.2069, -0.6207,
                    -0.7241, -0.6724, -0.1034,  0.1552, -1.0345, -0.2069,
                     0.1034,  0.1552, -0.8276, -0.5172, -0.3621,  0.0000,
                     0.0517,  0.1552, -0.5690, -0.9310,  0.2586, -0.4655,
                     0.0000,  0.3103, -0.5172, -0.2069, -0.5172,  0.2069,
                     1.0345,  7.2932,  0.5690,  1.8621,  3.9828,  2.2242,
                     1.3448,  1.8621, -0.3621,  0.7241, -1.5000,  1.3448,
                     0.3621,  0.1034,  1.2414, -0.5690,  0.4138,  0.4655,
                    -0.4655,  0.5172,  0.5172,  0.6207,  0.2069,  0.1034,
                    -0.0517, -0.0517,  0.5172,  0.5690, -0.2069,  0.0000,
                    -0.7241,  1.9655, -0.6207,  0.0000]],
                  grad_fn=<AliasBackward0>)

In [10]:

# # Step 3: Export Quantized Model
os.makedirs('quantized', exist_ok=True)
sim.export(path='quantized', filename_prefix='ref_solution_quantized', dummy_input=dummy_input)


2025-02-26 11:52:02,977 - Utils - INFO - successfully created onnx model with 88/100 node names updated
2025-02-26 11:52:03,003 - Quant - INFO - Layers excluded from quantization: []
2025-02-26 11:52:03,005 - Quant - WARNING - Quantsim export will stop exporting encodings for saving and loading in a future AIMET release.
To export encodings for saving and loading, use QuantizationSimModel's save_encodings_to_json() utility instead.


In [11]:
print(sim.model)

PreprocessedMobileNetV2(
  (mobilenet_v2): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): QuantizedConv2d(
          3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
          (param_quantizers): ModuleDict(
            (weight): QuantizeDequantize(shape=(), qmin=-128, qmax=127, symmetric=True)
            (bias): None
          )
          (input_quantizers): ModuleList(
            (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
          )
          (output_quantizers): ModuleList(
            (0): None
          )
        )
        (1): Identity()
        (2): QuantizedReLU(
          (param_quantizers): ModuleDict()
          (input_quantizers): ModuleList(
            (0): None
          )
          (output_quantizers): ModuleList(
            (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
          )
        )
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): 

In [12]:
sim.model.mobilenet_v2.features[0][0]

QuantizedConv2d(
  3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
  (param_quantizers): ModuleDict(
    (weight): QuantizeDequantize(shape=(), qmin=-128, qmax=127, symmetric=True)
    (bias): None
  )
  (input_quantizers): ModuleList(
    (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
  )
  (output_quantizers): ModuleList(
    (0): None
  )
)

In [13]:
sim.model(mug_image)

DequantizedTensor([[-0.3103,  0.3103, -0.9828, -0.6724, -0.2069, -0.6207,
                    -0.7241, -0.6724, -0.1034,  0.1552, -1.0345, -0.2069,
                     0.1034,  0.1552, -0.8276, -0.5172, -0.3621,  0.0000,
                     0.0517,  0.1552, -0.5690, -0.9310,  0.2586, -0.4655,
                     0.0000,  0.3103, -0.5172, -0.2069, -0.5172,  0.2069,
                     1.0345,  7.2932,  0.5690,  1.8621,  3.9828,  2.2242,
                     1.3448,  1.8621, -0.3621,  0.7241, -1.5000,  1.3448,
                     0.3621,  0.1034,  1.2414, -0.5690,  0.4138,  0.4655,
                    -0.4655,  0.5172,  0.5172,  0.6207,  0.2069,  0.1034,
                    -0.0517, -0.0517,  0.5172,  0.5690, -0.2069,  0.0000,
                    -0.7241,  1.9655, -0.6207,  0.0000]],
                  grad_fn=<AliasBackward0>)

In [14]:
helper.print_probablities_from_output(sim.model(mug_image), dsutils.GLOBAL_CLASSES)

Top-5 predictions for  on :
31 Cup                   90.8%
34 Bowl                   3.3%
35 Banana                 0.6%
61 Vase                   0.4%
37 Orange                 0.4%


In [8]:
# Compile to TensorFlow Lite

import qai_hub as hub

compile_job = hub.submit_compile_job(
    model="reference_solution_quantized.aimet",
    device=hub.Device("Samsung Galaxy S24 (Family)"),
)
assert isinstance(compile_job, hub.CompileJob)

# Compile to a QNN Model Library
compile_job_qnn = hub.submit_compile_job(
    model="reference_solution_quantized.aimet",
    device=hub.Device("Samsung Galaxy S24 (Family)"),
    options="--target_runtime qnn_lib_aarch64_android --quantize_full_type int8",
)
assert isinstance(compile_job_qnn, hub.CompileJob)

Uploading reference_solution_quantized.aimet.zip
2025-02-26 11:59:47,382 - root - INFO - Uploading asset to https://tetrahub-qprod-userdata.s3-accelerate.amazonaws.com/models/mn068z98n_AXSAzw8ZnA18ydUt.aimet.zip?uploadId=fJXpEztz2HWtchFIee37UgsAVKehi9yP1IfKf8abuWfTbiKowtDYLBTg3KyJpST2YbXUJ_VefJqYxc4C75vvJDCQuJw46Akafb4kx2j69RhkJkhEGZwOQcpiwVrhhVd_MM07tB2a3aDpQNzzt0jpvotsGPNFcxC9n4p6ztAFMJ8-&partNumber=1&A


100%|██████████| 3.46M/3.46M [00:02<00:00, 1.74MB/s]

2025-02-26 11:59:49,474 - root - INFO - Successfully uploaded asset with response status: 200


Scheduled compile job (jpyzv4erg) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jpyzv4erg/

Uploading reference_solution_quantized.aimet.zip
2025-02-26 11:59:51,657 - root - INFO - Uploading asset to https://tetrahub-qprod-userdata.s3-accelerate.amazonaws.com/models/mqy1pzwrq_XouymYO04J5xUKB4.aimet.zip?uploadId=JXasc.YmZRzTPuPLzPBFB_rQFXP4GQ_rmIDvC4lUT8W3Frxvg1uo9FpUExpW825JSAuHWYjI7oroz8LUexUmeLeuPgAjmaiFanU6mMDs9h_oAY4r9mUoMnCj4Rm7wwHXMZame4eUfyzyXrSXy.4483av7hUrZqZMa_2_ZViMsyU-&partNumber=1&A


100%|██████████| 3.46M/3.46M [00:02<00:00, 1.73MB/s]

2025-02-26 11:59:53,753 - root - INFO - Successfully uploaded asset with response status: 200


Scheduled compile job (jp04e1y2g) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp04e1y2g/



In [9]:
profile_job     = hub.submit_profile_job(compile_job.get_target_model(), device=hub.Device("Snapdragon 8 Elite QRD"),)

profile_job_qnn = hub.submit_profile_job(compile_job_qnn.get_target_model(), device=hub.Device("Snapdragon 8 Elite QRD"),)

Scheduled profile job (jgkvrlzyp) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jgkvrlzyp/

Scheduled profile job (j5q09787p) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/j5q09787p/



In [ ]:
model_traced_original = torch.jit.trace(modelOriginal, dummy_input)



compile_job_original = hub.submit_compile_job(
    model=model_traced_original,
    device=hub.Device("Samsung Galaxy S24 (Family)"),
    input_specs=dict(image=input_shape)
)
assert isinstance(compile_job_original, hub.CompileJob)

# Compile to a QNN Model Library
compile_job_original_qnn = hub.submit_compile_job(
    model=model_traced_original,
    device=hub.Device("Samsung Galaxy S24 (Family)"),
    input_specs=dict(image=input_shape),
    options="--target_runtime qnn_lib_aarch64_android",
)
assert isinstance(compile_job_original_qnn, hub.CompileJob)

Uploading tmp4pf1vaxn.pt
2025-02-26 12:14:01,797 - root - INFO - Uploading asset to https://tetrahub-qprod-userdata.s3-accelerate.amazonaws.com/models/mqe8lrgvq_UbyJbnu52Lbvwv7x.pt?uploadId=vl73_sWooq3J7zrf68Q1NxOpTm6qtNuAOeLIMVkyu9se3C4ZAHlbpQTyR9CfoDZwANt90PkcbCcrYgac1kBc0EwiVfaokgYqisTJEcHQjb1BMy_E7kMFaC6l46oL81.B30tuPCiJmErW0KLGVt6SVvOw8dEmHVW_41917Zn.R0E-&partNumber=1&AWSAcces


100%|██████████| 9.37M/9.37M [00:02<00:00, 3.99MB/s]

2025-02-26 12:14:04,260 - root - INFO - Successfully uploaded asset with response status: 200


Scheduled compile job (j5mevw2qp) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/j5mevw2qp/

Uploading tmpbi1vprnt.pt
2025-02-26 12:14:06,315 - root - INFO - Uploading asset to https://tetrahub-qprod-userdata.s3-accelerate.amazonaws.com/models/mnzok56dm_gzFsP1EzMDDg71Uw.pt?uploadId=QeX9kBCpvLj9ly_sdvRtzX90qnF0qa.K0QxwTFfqhmfOsB.RJW0UCzUOsQlOC.3Uy.wRIxmjgb_bBY4ygi7fS3sNlyXyQgvGEMXPbawYvmcJgi_ffuLhcSc39Yod6f3HJe2VedEz5OgJXm00FhPhRrVhPjTAStHscvmctLlPrdc-&partNumber=1&AWSAcces


100%|██████████| 9.37M/9.37M [00:01<00:00, 5.59MB/s]

2025-02-26 12:14:08,075 - root - INFO - Successfully uploaded asset with response status: 200


Scheduled compile job (jgn0r9ym5) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jgn0r9ym5/



In [13]:
profile_job_original     = hub.submit_profile_job(compile_job_original.get_target_model(), device=hub.Device("Snapdragon 8 Elite QRD"),)

profile_job_original_qnn = hub.submit_profile_job(compile_job_original_qnn.get_target_model(), device=hub.Device("Snapdragon 8 Elite QRD"),)

Scheduled profile job (jp2x376mg) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp2x376mg/

Scheduled profile job (jpyzv4w4g) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jpyzv4w4g/



In [24]:
quanTFLITE = profile_job.download_results('profile_results')
quanQNN    = profile_job_qnn.download_results('profile_results')

Saved profile results to /home/centar15-desktop1/LPCV_2025_T1/src/reference_solution/profile_results/job_jpyzv4erg_optimized_tflite_jgkvrlzyp_results_12.23.47.701037.json
Saved profile results to /home/centar15-desktop1/LPCV_2025_T1/src/reference_solution/profile_results/job_jp04e1y2g_optimized_so_j5q09787p_results.json


In [23]:
quanTFLITE

ProfileJobResult
----------------
status                        : JobStatus
---------
code    : SUCCESS
message : 

url                           : https://app.aihub.qualcomm.com/jobs/jgkvrlzyp/
artifacts_dir                 : /home/centar15-desktop1/LPCV_2025_T1/src/reference_solution/profile_results
Estimated Inference Time (ms) : 0.269
Load Time (ms)                : 235.482
Peak Memory (MB)              : 129.34375
Compute Units (layers)        : NPU: 74

In [25]:
quanQNN

ProfileJobResult
----------------
status                        : JobStatus
---------
code    : SUCCESS
message : 

url                           : https://app.aihub.qualcomm.com/jobs/j5q09787p/
artifacts_dir                 : /home/centar15-desktop1/LPCV_2025_T1/src/reference_solution/profile_results
Estimated Inference Time (ms) : 0.206
Load Time (ms)                : 201.704
Peak Memory (MB)              : 179.52734375
Compute Units (layers)        : NPU: 70

In [26]:
quanTFLITE_original = profile_job_original.download_results('profile_results')
quanQNN_original    = profile_job_original_qnn.download_results('profile_results')

Saved profile results to /home/centar15-desktop1/LPCV_2025_T1/src/reference_solution/profile_results/job_j5mevw2qp_optimized_tflite_jp2x376mg_results.json
Saved profile results to /home/centar15-desktop1/LPCV_2025_T1/src/reference_solution/profile_results/job_jgn0r9ym5_optimized_so_jpyzv4w4g_results.json


In [27]:
quanTFLITE_original

ProfileJobResult
----------------
status                        : JobStatus
---------
code    : SUCCESS
message : 

url                           : https://app.aihub.qualcomm.com/jobs/jp2x376mg/
artifacts_dir                 : /home/centar15-desktop1/LPCV_2025_T1/src/reference_solution/profile_results
Estimated Inference Time (ms) : 0.39
Load Time (ms)                : 365.974
Peak Memory (MB)              : 230.34375
Compute Units (layers)        : NPU: 70

In [28]:
quanQNN_original

ProfileJobResult
----------------
status                        : JobStatus
---------
code    : SUCCESS
message : 

url                           : https://app.aihub.qualcomm.com/jobs/jpyzv4w4g/
artifacts_dir                 : /home/centar15-desktop1/LPCV_2025_T1/src/reference_solution/profile_results
Estimated Inference Time (ms) : 0.394
Load Time (ms)                : 321.29
Peak Memory (MB)              : 176.10546875
Compute Units (layers)        : NPU: 103

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

validationDataloader = DataLoader(datasetCOCO, batch_size = 64, shuffle = False)
with torch.no_grad():
    print("Original  model accuracy = ", helper.check_accuracy(modelOriginal.cuda(), validationDataloader, device))
    # print("Quantized model accuracy = ", helper.check_accuracy(sim.model.cuda(), validationDataloader, device))

Original  model accuracy =  0.3803228398217807


In [16]:
with torch.no_grad():
    #print("Original  model accuracy = ", helper.check_accuracy(modelOriginal.cuda(), validationDataloader, device))
    print("Quantized model accuracy = ", helper.check_accuracy(sim.model.cuda(), validationDataloader, device))

Quantized model accuracy =  0.24775399897743042
